In [5]:
import numpy as np
import xarray as xr
import pandas as pd
import datetime


from datetime import timedelta, datetime, date
import calendar

import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import os
import glob


In [6]:
from dask.distributed import Client, LocalCluster

cluster = LocalCluster()
client = Client(cluster)
cluster

In [2]:
!which python

/local/ERAin/bin/python


In [3]:
xr.__version__

'0.14.0'

# HEre below I print out my directory tree, if yours is similar, you might not need to do anything

In [4]:
!ls /Data2/SubX/EMC/GEFS/

cape					   sty_sfc
climatology				   swe_sfc
correctedfiles.d			   ta_10
cprat_sfc				   ta_100
DJF					   ta_30
dlwrf_sfc				   ta_50
dswrf_sfc				   tas_2m
forecast				   tasmax_sfc
grb2_databefore1strecord.txt		   tasmin_sfc
grb2_large_int_byte_files.txt		   tdps_sfc
hfls_sfc				   ts_sfc
hfss_sfc				   ua_10
hlcy					   ua_100
huss_850				   ua_10m
JJA					   ua_200
originaltarfiles.d			   ua_30
pentad_climatology			   ua_50
prate_sfc				   ua_850
pr_sfc					   ulwrf_sfc
psl_sfc					   uswrf_sfc
rlut_toa				   va_10
RMM					   va_100
ROMI					   va_10m
sic_sfc					   va_200
snc_sfc					   va_30
snod_sfc				   va_50
soilw1_sfc				   va_850
soilw2_sfc				   wap_500
soilw3_sfc				   zg_10
soilw4_sfc				   zg_200
SON					   zg_30
sorted_grb2_EMC_GEFS_prob_files_final.txt  zg_50
sorted_grb2_prob_files.txt		   zg_500
sort_grb2_large_int_byte_files.txt	   zg_850
stx_sfc


In [6]:
!ls /Data2/SubX/EMC/GEFS/tdps_sfc/


1999  2001  2003  2005	2007  2009  2011  2013	2015
2000  2002  2004  2006	2008  2010  2012  2014	2016


In [7]:
!ls /Data2/SubX/EMC/GEFS/tdps_sfc/1999

01  02	03  04	05  06	07  08	09  10	11  12


In [8]:
!ls /Data2/SubX/EMC/GEFS/tdps_sfc/1999/01

tdps_sfc_GEFS_06jan1999_00z_d01_d35_m00.grb2
tdps_sfc_GEFS_06jan1999_00z_d01_d35_m01.grb2
tdps_sfc_GEFS_06jan1999_00z_d01_d35_m02.grb2
tdps_sfc_GEFS_06jan1999_00z_d01_d35_m03.grb2
tdps_sfc_GEFS_06jan1999_00z_d01_d35_m04.grb2
tdps_sfc_GEFS_06jan1999_00z_d01_d35_m05.grb2
tdps_sfc_GEFS_06jan1999_00z_d01_d35_m06.grb2
tdps_sfc_GEFS_06jan1999_00z_d01_d35_m07.grb2
tdps_sfc_GEFS_06jan1999_00z_d01_d35_m08.grb2
tdps_sfc_GEFS_06jan1999_00z_d01_d35_m09.grb2
tdps_sfc_GEFS_06jan1999_00z_d01_d35_m10.grb2
tdps_sfc_GEFS_13jan1999_00z_d01_d35_m00.grb2
tdps_sfc_GEFS_13jan1999_00z_d01_d35_m01.grb2
tdps_sfc_GEFS_13jan1999_00z_d01_d35_m02.grb2
tdps_sfc_GEFS_13jan1999_00z_d01_d35_m03.grb2
tdps_sfc_GEFS_13jan1999_00z_d01_d35_m04.grb2
tdps_sfc_GEFS_13jan1999_00z_d01_d35_m05.grb2
tdps_sfc_GEFS_13jan1999_00z_d01_d35_m06.grb2
tdps_sfc_GEFS_13jan1999_00z_d01_d35_m07.grb2
tdps_sfc_GEFS_13jan1999_00z_d01_d35_m08.grb2
tdps_sfc_GEFS_13jan1999_00z_d01_d35_m09.grb2
tdps_sfc_GEFS_13jan1999_00z_d01_d35_m10.grb2
tdps_sfc_G

# This is my preprocessing function, which I run on each grib file

In [9]:
def ppf2_filename(ds,filename):
    # rename coordinates
    ds  = ds.rename({ 'lon_0':'lon', 'lat_0':'lat', 'forecast_time0':'L'})
    # getting start time from file name
    ts = filename.split('_')
    whereGEFS = ts.index('GEFS')
    dateSstr = ts[whereGEFS+1]
    dateS = datetime.strptime(dateSstr, '%d%b%Y')
    ds.coords['S'] = 'S', np.atleast_1d(dateS)
    
    # replacing L dates with integer (lead days)
    # so that I can reduce to a fixed size
    # BEWARE - this is a hardcoded step now, so it needs to be checked for each model
    
    #if you comment in the two print lines, you will see how L is read by xarray from the grib file
#     print(ds.L)
#     print(ds.S)
    ds.coords['L']=np.arange(0.5,35.5)

    # select US
    ds = ds.sel(lat=slice(42,32),lon=slice(-103+360,-90+360))
#     print(ds)
    return ds

L in this files is read by xarray like this

`<xarray.DataArray 'L' (L: 35)>
array([  3600000000000,   7200000000000,  10800000000000,  14400000000000,
        18000000000000,  21600000000000,  25200000000000,  28800000000000,
        32400000000000,  36000000000000,  39600000000000,  43200000000000,
        46800000000000,  50400000000000,  54000000000000,  57600000000000,
        61200000000000,  64800000000000,  68400000000000,  72000000000000,
        75600000000000,  79200000000000,  82800000000000,  86400000000000,
        90000000000000,  93600000000000,  97200000000000, 100800000000000,
       104400000000000, 108000000000000, 111600000000000, 115200000000000,
       118800000000000, 122400000000000, 126000000000000],
      dtype='timedelta64[ns]')`

however if I use wgrib2 on one file 

`!wgrib2 -v /Data2/SubX/EMC/GEFS/cape/1999/06/cape_GEFS_02jun1999_00z_d01_d35_m00.grb2 `


I get something like

` 1:0:d=1999060200:CAPE Convective Available Potential Energy [J/kg]:180-0 mb above ground:1 day fcst:ENS=low-res ctl
2:46954:d=1999060200:CAPE Convective Available Potential Energy [J/kg]:180-0 mb above ground:2 day fcst:ENS=low-res ctl
3:92924:d=1999060200:CAPE Convective Available Potential Energy [J/kg]:180-0 mb above ground:3 day fcst:ENS=low-res ctl
4:139690:d=1999060200:CAPE Convective Available Potential Energy [J/kg]:180-0 mb above ground:4 day fcst:ENS=low-res ctl
5:186582:d=1999060200:CAPE Convective Available Potential Energy [J/kg]:180-0 mb above ground:5 day fcst:ENS=low-res ctl `

Let's make sure I am correctly overwriting L.

I am now using L=[0.5, 1.5, 2.5, ... , 35.5]

# Load, preprocess, append, and concatenate all the grib files. In this notebook I ran it on half of the data available.

### I will comment the code below for pr_sfc. I repeat the code for each variable separiately. I could loop it for a list of varname =[ 'pr_sfc' , ....] but I preferred keeping it separate.

In [10]:
varname = 'pr_sfc'
ds_Y = []
for iy in np.arange(1999, 2017):
    print( iy)
    ds_M = []
    for ensnum in np.arange(0,11):
        # JUNE files
        mylist = glob.glob('/Data2/SubX/EMC/GEFS/'+varname+'/'+np.str(iy)+'/06/*_m'+np.str(ensnum).zfill(2) +'.grb2') 
        
        # list of one file for each S
        pr_S_list = []
        for ifilename in mylist:
            # open file
            dstemp = xr.open_dataset(ifilename, engine='pynio')
            # define M value
            dstemp.coords['M'] = 'M', np.atleast_1d(ensnum)
            # preprocess
            dstemp1 = ppf2_filename(dstemp, ifilename)
            del dstemp
            # append file for each S
            pr_S_list.append(dstemp1)
            del dstemp1
        # concatenate along S
        pr_S = xr.concat(pr_S_list, dim='S')
        # JULY files
        mylist = glob.glob('/Data2/SubX/EMC/GEFS/'+varname+'/'+np.str(iy)+'/07/*_m'+np.str(ensnum).zfill(2) +'.grb2') 
        pr_S_list = []
        for ifilename in mylist:
            dstemp = xr.open_dataset(ifilename, engine='pynio')
            dstemp.coords['M'] = 'M', np.atleast_1d(ensnum)
            dstemp1 = ppf2_filename(dstemp, ifilename)
            del dstemp
            pr_S_list.append(dstemp1)
            del dstemp1
        pr_S = pr_S.merge(xr.concat(pr_S_list, dim='S'))
        
        # AUGUST files
        mylist = glob.glob('/Data2/SubX/EMC/GEFS/'+varname+'/'+np.str(iy)+'/08/*_m'+np.str(ensnum).zfill(2) +'.grb2') 
        pr_S_list = []
        for ifilename in mylist:
            dstemp = xr.open_dataset(ifilename, engine='pynio')
            dstemp.coords['M'] = 'M', np.atleast_1d(ensnum)
            dstemp1 = ppf2_filename(dstemp, ifilename)
            del dstemp
            pr_S_list.append(dstemp1)
            del dstemp1
        pr_S = pr_S.merge(xr.concat(pr_S_list, dim='S'))
        ds_M.append(pr_S)
        del pr_S
    ds_all = xr.concat(ds_M, dim='M')
    del ds_M
    ds_Y.append(ds_all)
del ds_all
ds_allpr = xr.concat(ds_Y, dim='S')
del ds_Y
         
        
        


1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016


In [11]:
varname = 'tas_2m'
ds_Y = []
for iy in np.arange(1999, 2017):
    print( iy)
    ds_M = []
    for ensnum in np.arange(0,11):
        mylist = glob.glob('/Data2/SubX/EMC/GEFS/'+varname+'/'+np.str(iy)+'/06/*_m'+np.str(ensnum).zfill(2) +'.grb2') 
        
        pr_S_list = []
        for ifilename in mylist:
            dstemp = xr.open_dataset(ifilename, engine='pynio')
            dstemp.coords['M'] = 'M', np.atleast_1d(ensnum)
            dstemp1 = ppf2_filename(dstemp, ifilename)
    #         print( dstemp1)
            del dstemp
            pr_S_list.append(dstemp1)
            del dstemp1
        pr_S = xr.concat(pr_S_list, dim='S')
    #     ds_M.append(pr_S)
    #     del pr_S
        mylist = glob.glob('/Data2/SubX/EMC/GEFS/'+varname+'/'+np.str(iy)+'/07/*_m'+np.str(ensnum).zfill(2) +'.grb2') 
#         print(len(mylist))
        pr_S_list = []
        for ifilename in mylist:
            dstemp = xr.open_dataset(ifilename, engine='pynio')
            dstemp.coords['M'] = 'M', np.atleast_1d(ensnum)
            dstemp1 = ppf2_filename(dstemp, ifilename)
    #         print( dstemp1)
            del dstemp
            pr_S_list.append(dstemp1)
            del dstemp1
        pr_S = pr_S.merge(xr.concat(pr_S_list, dim='S'))
    #     pr_S = xr.concat(pr_S_list, dim='S')
    #     ds_M.append(pr_S)
    #     del pr_S
        mylist = glob.glob('/Data2/SubX/EMC/GEFS/'+varname+'/'+np.str(iy)+'/08/*_m'+np.str(ensnum).zfill(2) +'.grb2') 
#         print(len(mylist))
        pr_S_list = []
        for ifilename in mylist:
            dstemp = xr.open_dataset(ifilename, engine='pynio')
            dstemp.coords['M'] = 'M', np.atleast_1d(ensnum)
            dstemp1 = ppf2_filename(dstemp, ifilename)
    #         print( dstemp1)
            del dstemp
            pr_S_list.append(dstemp1)
            del dstemp1
        pr_S = pr_S.merge(xr.concat(pr_S_list, dim='S'))
    #     pr_S = xr.concat(pr_S_list, dim='S')
        ds_M.append(pr_S)
        del pr_S
    ds_all = xr.concat(ds_M, dim='M')
    del ds_M
    ds_Y.append(ds_all)
del ds_all
ds_alltas_2m = xr.concat(ds_Y, dim='S')
del ds_Y
         
        
        
#     pr_S=xr.open_mfdataset(mylist, concat_dim='S', preprocess=ppf2, autoclose=True, engine='pynio')
#     pr_S.coords['M'] = 'M', np.atleast_1d(ensnum)
#     print( pr_S)

1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016


In [12]:
varname = 'tas_2m'
ds_Y = []
for iy in np.arange(1999, 2017):
    print( iy)
    ds_M = []
    for ensnum in np.arange(0,11):
        mylist = glob.glob('/Data2/SubX/EMC/GEFS/'+varname+'/'+np.str(iy)+'/06/*_m'+np.str(ensnum).zfill(2) +'.grb2') 
#         print(len(mylist))
        pr_S_list = []
        for ifilename in mylist:
            dstemp = xr.open_dataset(ifilename, engine='pynio')
            dstemp.coords['M'] = 'M', np.atleast_1d(ensnum)
            dstemp1 = ppf2_filename(dstemp, ifilename)
    #         print( dstemp1)
            del dstemp
            pr_S_list.append(dstemp1)
            del dstemp1
        pr_S = xr.concat(pr_S_list, dim='S')
    #     ds_M.append(pr_S)
    #     del pr_S
        mylist = glob.glob('/Data2/SubX/EMC/GEFS/'+varname+'/'+np.str(iy)+'/07/*_m'+np.str(ensnum).zfill(2) +'.grb2') 
#         print(len(mylist))
        pr_S_list = []
        for ifilename in mylist:
            dstemp = xr.open_dataset(ifilename, engine='pynio')
            dstemp.coords['M'] = 'M', np.atleast_1d(ensnum)
            dstemp1 = ppf2_filename(dstemp, ifilename)
    #         print( dstemp1)
            del dstemp
            pr_S_list.append(dstemp1)
            del dstemp1
        pr_S = pr_S.merge(xr.concat(pr_S_list, dim='S'))
    #     pr_S = xr.concat(pr_S_list, dim='S')
    #     ds_M.append(pr_S)
    #     del pr_S
        mylist = glob.glob('/Data2/SubX/EMC/GEFS/'+varname+'/'+np.str(iy)+'/08/*_m'+np.str(ensnum).zfill(2) +'.grb2') 
#         print(len(mylist))
        pr_S_list = []
        for ifilename in mylist:
            dstemp = xr.open_dataset(ifilename, engine='pynio')
            dstemp.coords['M'] = 'M', np.atleast_1d(ensnum)
            dstemp1 = ppf2_filename(dstemp, ifilename)
    #         print( dstemp1)
            del dstemp
            pr_S_list.append(dstemp1)
            del dstemp1
        pr_S = pr_S.merge(xr.concat(pr_S_list, dim='S'))
    #     pr_S = xr.concat(pr_S_list, dim='S')
        ds_M.append(pr_S)
        del pr_S
    ds_all = xr.concat(ds_M, dim='M')
    del ds_M
    ds_Y.append(ds_all)
del ds_all
ds_alltas_2m = xr.concat(ds_Y, dim='S')
del ds_Y
         
        
        
#     pr_S=xr.open_mfdataset(mylist, concat_dim='S', preprocess=ppf2, autoclose=True, engine='pynio')
#     pr_S.coords['M'] = 'M', np.atleast_1d(ensnum)
#     print( pr_S)

1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016


In [13]:

varname = 'tdps_sfc'
ds_Y = []
for iy in np.arange(1999, 2017):
    print( iy)
    ds_M = []
    for ensnum in np.arange(0,11):
        mylist = glob.glob('/Data2/SubX/EMC/GEFS/'+varname+'/'+np.str(iy)+'/06/*_m'+np.str(ensnum).zfill(2) +'.grb2') 
#         print(len(mylist))
        pr_S_list = []
        for ifilename in mylist:
            dstemp = xr.open_dataset(ifilename, engine='pynio')
            dstemp.coords['M'] = 'M', np.atleast_1d(ensnum)
            dstemp1 = ppf2_filename(dstemp, ifilename)
    #         print( dstemp1)
            del dstemp
            pr_S_list.append(dstemp1)
            del dstemp1
        pr_S = xr.concat(pr_S_list, dim='S')
    #     ds_M.append(pr_S)
    #     del pr_S
        mylist = glob.glob('/Data2/SubX/EMC/GEFS/'+varname+'/'+np.str(iy)+'/07/*_m'+np.str(ensnum).zfill(2) +'.grb2') 
#         print(len(mylist))
        pr_S_list = []
        for ifilename in mylist:
            dstemp = xr.open_dataset(ifilename, engine='pynio')
            dstemp.coords['M'] = 'M', np.atleast_1d(ensnum)
            dstemp1 = ppf2_filename(dstemp, ifilename)
    #         print( dstemp1)
            del dstemp
            pr_S_list.append(dstemp1)
            del dstemp1
        pr_S = pr_S.merge(xr.concat(pr_S_list, dim='S'))
    #     pr_S = xr.concat(pr_S_list, dim='S')
    #     ds_M.append(pr_S)
    #     del pr_S
        mylist = glob.glob('/Data2/SubX/EMC/GEFS/'+varname+'/'+np.str(iy)+'/08/*_m'+np.str(ensnum).zfill(2) +'.grb2') 
#         print(len(mylist))
        pr_S_list = []
        for ifilename in mylist:
            dstemp = xr.open_dataset(ifilename, engine='pynio')
            dstemp.coords['M'] = 'M', np.atleast_1d(ensnum)
            dstemp1 = ppf2_filename(dstemp, ifilename)
    #         print( dstemp1)
            del dstemp
            pr_S_list.append(dstemp1)
            del dstemp1
        pr_S = pr_S.merge(xr.concat(pr_S_list, dim='S'))
    #     pr_S = xr.concat(pr_S_list, dim='S')
        ds_M.append(pr_S)
        del pr_S
    ds_all = xr.concat(ds_M, dim='M')
    del ds_M
    ds_Y.append(ds_all)
del ds_all
ds_alltdps_2m = xr.concat(ds_Y, dim='S')
del ds_Y
         
        
        
#     pr_S=xr.open_mfdataset(mylist, concat_dim='S', preprocess=ppf2, autoclose=True, engine='pynio')
#     pr_S.coords['M'] = 'M', np.atleast_1d(ensnum)
#     print( pr_S)

1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016


In [14]:

varname = 'cape'
ds_Y = []
for iy in np.arange(1999, 2017):
    print( iy)
    ds_M = []
    for ensnum in np.arange(0,11):
        mylist = glob.glob('/Data2/SubX/EMC/GEFS/'+varname+'/'+np.str(iy)+'/06/*_m'+np.str(ensnum).zfill(2) +'.grb2') 
#         print(len(mylist))
        pr_S_list = []
        for ifilename in mylist:
            dstemp = xr.open_dataset(ifilename,  engine='pynio')
            dstemp.coords['M'] = 'M', np.atleast_1d(ensnum)
            dstemp1 = ppf2_filename(dstemp, ifilename)
    #         print( dstemp1)
            del dstemp
            pr_S_list.append(dstemp1)
            del dstemp1
        pr_S = xr.concat(pr_S_list, dim='S')
    #     ds_M.append(pr_S)
    #     del pr_S
        mylist = glob.glob('/Data2/SubX/EMC/GEFS/'+varname+'/'+np.str(iy)+'/07/*_m'+np.str(ensnum).zfill(2) +'.grb2') 
#         print(len(mylist))
        pr_S_list = []
        for ifilename in mylist:
            dstemp = xr.open_dataset(ifilename, engine='pynio')
            dstemp.coords['M'] = 'M', np.atleast_1d(ensnum)
            dstemp1 = ppf2_filename(dstemp, ifilename)
    #         print( dstemp1)
            del dstemp
            pr_S_list.append(dstemp1)
            del dstemp1
        pr_S = pr_S.merge(xr.concat(pr_S_list, dim='S'))
    #     pr_S = xr.concat(pr_S_list, dim='S')
    #     ds_M.append(pr_S)
    #     del pr_S
        mylist = glob.glob('/Data2/SubX/EMC/GEFS/'+varname+'/'+np.str(iy)+'/08/*_m'+np.str(ensnum).zfill(2) +'.grb2') 
#         print(len(mylist))
        pr_S_list = []
        for ifilename in mylist:
            dstemp = xr.open_dataset(ifilename, engine='pynio')
            dstemp.coords['M'] = 'M', np.atleast_1d(ensnum)
            dstemp1 = ppf2_filename(dstemp, ifilename)
    #         print( dstemp1)
            del dstemp
            pr_S_list.append(dstemp1)
            del dstemp1
        pr_S = pr_S.merge(xr.concat(pr_S_list, dim='S'))
    #     pr_S = xr.concat(pr_S_list, dim='S')
        ds_M.append(pr_S)
        del pr_S
    ds_all = xr.concat(ds_M, dim='M')
    del ds_M
    ds_Y.append(ds_all)
del ds_all
ds_allcape = xr.concat(ds_Y, dim='S')
del ds_Y
         
        
        
#     pr_S=xr.open_mfdataset(mylist, concat_dim='S', preprocess=ppf2, autoclose=True, engine='pynio')
#     pr_S.coords['M'] = 'M', np.atleast_1d(ensnum)
#     print( pr_S)

1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016


# MERGE THEM IN ONE DATAARRAY

In [15]:
ds_allpr =  xr.merge([ds_allpr,ds_alltas_2m,ds_alltdps_2m, ds_allcape])

In [16]:
del ds_allcape
del ds_alltdps_2m
del ds_alltas_2m

In [17]:
ds_allpr.to_netcdf('ds_pr_t_td_cape_EMCC.nc')